In [1]:
import matplotlib
matplotlib.use('Agg')

#%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

from matplotlib import pyplot as plt
import scipy
from scipy.ndimage import gaussian_filter


import csv

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import os

import scipy.misc
#import skimage.filter

import parmap
import glob

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

from PIL import Image

    
clrs = ['red','blue','cyan','green','yellow','pink','magenta','black','lightgreen','lightblue']


Autosaving every 180 seconds


In [9]:
# ROTATE IMAGE
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# FIND ROTATION 
def find_rotation(locs,
                  frame,
                  plotting
                 ):  
    
    # This is the order that you should search for the centre of the animal; do not use median/mean; 
    centre_list = [7,6,5,8,4,9,3,10,2,11,1,12,0,13] # SPINE 2 - feautre 6 is centre of animal now;
    
    #
    padding_size = 400
    #if rgb==True:
    frame_padded = np.zeros((1024+padding_size*2, 1280+padding_size*2, 3), 'uint8')
    #else:
    #    frame_padded = np.zeros((1024+padding_size*2, 1280+padding_size*2), 'uint8')
        
    #print (frame_padded.shape, frame.shape)
    frame_padded[padding_size:padding_size+1024, padding_size:padding_size+1280] = frame
    #frame = np.pad(frame, 2, pad_with, padder=np.mean(frame))
    
    frame = frame_padded.copy()
    locs+=padding_size
    
    # how wide is the final image of the animal
    width = 100
    
    # temporary padding to help with rotation;
    padding = 150
       
    if False:
        centrex = int(np.nanmean(locs[:,1]))
        centrey = int(np.nanmean(locs[:,0]))
    else:
        # loop through all the feature from spine going outwards
        for k in range(len(centre_list)):   
            temp = locs[centre_list[k],0]
            if np.isnan(temp)==False:
                break

        centrey = int(locs[centre_list[k],0])
        centrex = int(locs[centre_list[k],1])
    
    # GRAB CENTRED PATCH
    frame_patch = frame[centrex-width-padding:centrex+width+padding, 
                        centrey-width-padding:centrey+width+padding]#[:,:,1] #GRAB SINGLE CHAN COLOR
    
    # ADD LABELS
    if plotting:
        size = 6
        frame_image = frame_patch.copy()
        for k in range(locs.shape[0]):
            temp = locs[k,0]
            if np.isnan(temp)==True:
                continue
            tempy = int(locs[k,0])-centrey-padding-width
            tempx = int(locs[k,1])-centrex-padding-width
            frame_image[tempx-size: tempx+size,
                        tempy-size: tempy+size]= np.nan
    else:
        frame_image = None

    # EXCLUDE PATCHES THAT ARE NOT RIGHT SIZE DUE TO CLIPPING;
    # TODO: fix this so it is not required
    if frame_patch.shape[0]!= (width*2 + padding*2):
        return None
    
    if frame_patch.shape[1]!= (width*2 + padding*2):
        return None

    # ROTATE RAW DATA
    try:

        # FIND CENTRES
        idx = np.where(np.isnan(locs[:,0]))[0]
        locs = np.delete(locs, idx, 0)
    
        # FIND ROTATION USING PCA
        x = locs[:,0]
        y = locs[:,1]
        x = x - np.mean(x)
        y = y - np.mean(y)
        coords = np.vstack([x, y])
        if coords.shape[1]==1:
            frame_patch = frame_patch[padding:-padding, padding: -padding]
            return frame_patch, frame_image            
            
        cov = np.cov(coords)
        evals, evecs = np.linalg.eig(cov)

        sort_indices = np.argsort(evals)[::-1]
        x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
        x_v2, y_v2 = evecs[:, sort_indices[1]]
        theta = np.arctan((x_v1)/(y_v1))  
    
        # ROTATE IMAGE
        frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
        frame_rotated = frame_rotated[padding:-padding, padding: -padding]
        
        if plotting:
            # rotate png image (OPTIONAL)
            frame_image = rotate_image(frame_image, -theta*180/3.14159)
            frame_image = frame_image[padding:-padding, padding: -padding]
         
    except:
        print ("Couldn't rotate image, returning original patch")
        print ("coords = np.vstack([x, y]): ", coords, coords.shape)
        
        
        return frame_patch, frame_image
        
    return frame_rotated, frame_image


def try_float(v):
   try:
       return float(v)
   except Exception:
       return np.nan

def load_xy(trace):
    loc= np.array([try_float(item) for item in trace[2:]])
    locx = loc[::2]
    locy = loc[1::2]
    loc = np.stack((locx,locy)).T
    
    return loc


            
            
  
def generate_training_image_sleap(frame_ids,
                                  video_name,
                                  root_dir,
                                  save_img,
                                  traces,
                                  rgb=False
                                  ):
    
    # OPEN VIDEO
    original_vid = cv2.VideoCapture(video_name)

    # RESET VID TO FIRST FRAME ID (frame_ids are in sequence)
    start = frame_ids[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    
    # FIND START FRAME IN SLEAP .CSV TRACES OUTPUT
    ctr_traces=0
    while True:
        frame_id2 =int(traces[ctr_traces][1])
        if frame_id2 >= start:
            break
        ctr_traces+=1
    
    
    # LOOP OVER ALL SELECTED FRAME_IDs
    for frame_id in frame_ids:
        ret, frame = original_vid.read()
        if rgb==False:
            green_chan = 1
            frame = frame[:,:,green_chan]
                
        # LOOP OVER ALL THE TRACKLETS IN THE FRAME
        while True:
            
            frame_id2 =int(traces[ctr_traces][1])
            track_id = int(traces[ctr_traces][0].replace('track_',''))
            if  frame_id2 != frame_id:
                break
            
            # Generate save filename
            out_fname = root_dir+str(track_id)+'/frame_'+str(frame_id2).zfill(7)+"_id_"+str(track_id)+'.npy'

            
            #print ("track_id: ", track_id, "   frame: ", frame_ids[ctr_ids])
            loc = load_xy(traces[ctr_traces])

            ##############################################################
            ###################### GET X,Y LOCS ##########################
            ##############################################################
            tempx = loc[:,0]
            tempy = loc[:,1]
            locs = np.vstack((tempx,tempy)).T

            # exit if all np.nan vals
            #if locs.shape[0]==0:
            #    ctr_traces+=1
            #    continue

            ##############################################################
            ################### ROTATE IMG ###############################
            ##############################################################
            frame_rotated, frame_image = find_rotation(locs,
                                                       frame,
                                                       plotting)

            ##############################################################
            ################### SAVE IMG AND NPY #########################
            ##############################################################
            # MAKE DIR IF NOT ALREADY MDE
            try:
                local_dir = root_dir+str(track_id)
                os.mkdir(local_dir)
            except:
                pass

            # SAVE IMAGE FOR MANUAL EVALUATION
            if save_img:
            
                #Rescale to 0-255 and convert to uint8
                rescaled = (255.0 / frame_image.max() * (frame_image - frame_image.min())).astype(np.uint8)

                im = Image.fromarray(rescaled)
                im.save(out_fname[:-4]+'.png')

            # SAVE DATA
            if frame_rotated.shape[0]!=200:
                print (frame_id, track_id, frame_rotated.shape)
            np.savez(out_fname[:-4]+'.npz', 
                     frame=frame_rotated,
                     locs = locs,
                     frame_id = frame_id,
                     track_id_sleap = track_id)
            
            # MOVE TO NEXT VALUE IN SLEAP TRACES
            ctr_traces+=1
            

            
def generate_CSUA_tracks_for_CNN_training_and_prediction(home_dir,
                                                         fname_tracks,
                                                         parallelize,
                                                         n_processes,
                                                         save_img):

    root_dir = home_dir + '/animal_images/'
    try:
        os.mkdir(root_dir)
    except:
        pass

    # load video fname
    video_name = home_dir +'/'+os.path.split(home_dir)[1]+'.avi'
    #print ("video name): ", video_name)
    # load tracks
    tracks = np.load(fname_tracks)
    track_ids = np.unique(tracks[:,0])
    
    fnames_complete = glob.glob(root_dir+"/*")
    #track_ids_split = np.array_split(track_ids, n_processes*10)
            

    # LOAD ALL THE ANIMALS IN THAT FRAME AND SAVE THEM IN DIR
    if parallelize:
        parmap.map(generate_training_image_sleap_per_track,
                       track_ids,
                       video_name,
                       root_dir,
                       save_img,
                       tracks,
                       pm_processes=n_processes,
                       pm_pbar=True)

    else:
        from tqdm import trange
        for k in trange(len(track_ids_split)):
            generate_training_image_sleap_per_track(
                       track_ids[k],
                       video_name,
                       root_dir,
                       save_img,
                       tracks)
               

# 
def generate_training_image_sleap_per_track(track_id,
                                            video_name,
                                            root_dir,
                                            save_img,
                                            tracks):
    
   # for track_id in track_ids:

    # find video frames for specific track:
    # print ('tracks ', tracks)
    # print ('track_id ', track_id)
    idx_track = np.where(tracks[:,0]==track_id)[0]

    # print ("track id: ", track_id, " has vals at: ", idx_track, idx_track.shape)
    frame_ids = tracks[idx_track][:,1]
    # print ('frame ids: ', frame_ids)

    # OPEN VIDEO
    #video_name = home_dir +'/'+os.path.split(home_dir)[1]+'.avi'
    original_vid = cv2.VideoCapture(video_name)

    # RESET VID TO FIRST FRAME ID (frame_ids are in sequence)
    start = frame_ids[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)


    # LOOP OVER ALL SELECTED FRAME_IDs
    data_array = []
    locs_array = []
    for ctr, frame_id in enumerate(frame_ids):
        ret, frame = original_vid.read()

        ##############################################################
        ###################### GET X,Y LOCS ##########################
        ##############################################################
        locs = tracks[idx_track[ctr]][2:]

        temp = np.zeros((locs.shape[0]//2,2))
        temp[:,0]=locs[::2]
        temp[:,1]=locs[1::2]
        locs = temp.copy()

        locs_array.append(locs)

        ##############################################################
        ################### ROTATE IMG ###############################
        ##############################################################
        '''  frame_rotoated is the 200 x 200 x 3 image;
             frame_image is the same + the feature locations;
        '''
        frame_rotated, frame_image = find_rotation(locs,
                                                   frame,
                                                   save_img)

        ##############################################################
        ################### SAVE IMG AND NPY #########################
        ##############################################################
        # MAKE DIR IF NOT ALREADY MDE
        try:
            os.mkdir(root_dir)
        except:
            pass

        try:
            local_dir = root_dir+str(int(track_id))
            os.mkdir(local_dir)
        except:
            pass

        # SAVE IMAGE FOR MANUAL EVALUATION
        if save_img:

            # rescale to 0-255 and convert to uint8
            rescaled = (255.0 / frame_image.max() * (frame_image - frame_image.min())).astype(np.uint8)

            im = Image.fromarray(rescaled)
            im.save(out_fname[:-4]+'.png')

        data_array.append(frame_rotated)


    np.savez(os.path.join(root_dir,str(int(track_id)),str(int(track_id))+'.npz'), 
             images = data_array,
             locs = locs_array,
             frame_id = frame_ids,
             track_id_sleap = track_id)

        
        

In [10]:
# ##################################################################
# ######### INSERT COMMAND LINE CODE TO CONVERT .slp TO .csv #######
# ##################################################################
# root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/'
# dirs = glob.glob(root_dir+"*")

# for dir_ in dirs:
#     #print ("INSERT .slp to .csv code here"; NOTE RTX 3080 may not work, but sleap should still install in CPU mode
#     fname_slp = os.path.join(dir_, os.path.split(dir_)[1]+'.avi.predictions.slp')
#     text = 'sleap-convert --format analysis ' + fname_slp # "/home/cat/Downloads/sleap_video.mp4.tracker_flow.slp"
#     print ("text: ", text)
#     os.system(text)
#     break
    

In [11]:
##################################################################
#########  .slp TO .csv #######
##################################################################

# import csv

# traces =[]
# with open('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/2020-3-9_12_14_22_815059_compressed/video.mp4.analysis.csv', mode='r') as infile:
#     reader = csv.reader(infile)
#     with open('coors_new.csv', mode='w') as outfile:
#         writer = csv.writer(outfile)
#         for row in reader:
#             traces.append(row)

# # REMOVE HEADER
# print ("HEADER: ", traces.pop(0))
# print('')
# # SHOW FIRST LINE OF DATA
# print (traces[0])



In [12]:
########################################################
######### EXTRACT IMAGES FOR EVERY ROW IN TRACES #######
########################################################
''' Function generates the images required for CNN prediction from the datasets
    Required:
    - .csv file (after slp_to_csv converstoin from sleap)
    - .avi video file    
'''
            
# 
home_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed' 
fname_tracks = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy'
parallelize = True
n_processes = 12
save_img=False

generate_CSUA_tracks_for_CNN_training_and_prediction(home_dir,
                                                     fname_tracks,
                                                     parallelize,
                                                     n_processes,
                                                     save_img)


2208it [18:31,  1.99it/s]                          


In [87]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy')
print (data.shape)
print (data)

(307652, 30)
[[0.0000000e+00 0.0000000e+00 3.1035138e+02 ...           nan
            nan           nan]
 [1.0000000e+00 0.0000000e+00           nan ... 6.0292316e+02
            nan           nan]
 [0.0000000e+00 1.0000000e+00 3.1043536e+02 ...           nan
            nan           nan]
 ...
 [2.1840000e+03 8.9986000e+04 4.5234628e+02 ... 5.8704840e+02
  3.2452319e+02 6.0508130e+02]
 [2.1870000e+03 8.9986000e+04 1.2435868e+03 ...           nan
            nan           nan]
 [2.1880000e+03 8.9986000e+04 3.7664651e+02 ... 5.8874902e+02
  2.3473035e+02 5.6668292e+02]]


In [55]:


from tqdm import tqdm
track_id = 749

root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'
video_name = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi'                                  
save_img = False

# First find # frames and grab traces
tracks = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy')
print ("Tracks: [track_ID, frame_ID, loc0_x, loc0_y, loc1_x, loc1_y,...] " )
print (tracks[0])
                 
generate_training_image_sleap_per_track(track_id,
                                        video_name,
                                        root_dir,
                                        save_img,
                                        tracks)
            
    

Tracks: [track_ID, frame_ID, loc0_x, loc0_y, loc1_x, loc1_y,...] 
[  0.        0.      310.35138 572.4292        nan       nan 328.01074
 592.1255        nan       nan 332.36166 620.2585  322.48322 642.2069
 344.07208 678.09973 352.18756 696.1015  374.36697 704.43665 432.44778
 724.13495 458.30075 742.2128        nan       nan       nan       nan
       nan       nan]


In [54]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/0/0.npz')

images  = data['images']
print (images.shape)

for k in range(10):
    ax=plt.subplot(2,5,k+1)
    plt.imshow(images[k])
plt.show()

(172, 200, 200, 3)


<ipython-input-54-28600a08b9cd>:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax=plt.subplot(2,5,k+1)
<ipython-input-54-28600a08b9cd>:9: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [26]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy')
print (data)

[[0.0000000e+00 0.0000000e+00 3.1035138e+02 ...           nan
            nan           nan]
 [1.0000000e+00 0.0000000e+00           nan ... 6.0292316e+02
            nan           nan]
 [0.0000000e+00 1.0000000e+00 3.1043536e+02 ...           nan
            nan           nan]
 ...
 [2.1840000e+03 8.9986000e+04 4.5234628e+02 ... 5.8704840e+02
  3.2452319e+02 6.0508130e+02]
 [2.1870000e+03 8.9986000e+04 1.2435868e+03 ...           nan
            nan           nan]
 [2.1880000e+03 8.9986000e+04 3.7664651e+02 ... 5.8874902e+02
  2.3473035e+02 5.6668292e+02]]


In [ ]:
##############################################################
################### MANUALLY MAKE TRAINING DATA ##############
##############################################################

# Q COULD THIS BE AUTOMATED IN PART USING CNN ITERATION?


In [17]:
##############################################################
############# ASSEMBLE TRAINING DATA FOR CNN #################
##############################################################
import glob
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/2020-3-9_12_14_22_815059_compressed/cnn_training_10mins/animals/'
animal = 'pup2_noshaved'

fnames = glob.glob(root_dir+animal+'/**/*',recursive = True)
print (len(fnames))

frames = []
for fname in fnames:
    data = np.load(fname)
    temp = data['frame']
    if temp.shape[0]!=200:
        continue

    frames.append(temp)
frames = np.array(frames)
print (frames.shape)

np.save(root_dir+'3.npy',frames)

2313
(2313, 200, 200, 3)


In [25]:
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/2020-3-9_12_14_22_815059_compressed/cnn_output_10mins/'
tracklet='38'
data = np.load(os.path.join(root_dir, tracklet,'pred.npy') )       
    
print (data)
y  = np.histogram(data,bins=np.arange(0,4))
plt.bar(y[1][:-1],y[0], 0.9)
plt.savefig('/home/cat/38.png')

[0 2 2 ... 3 3 0]


In [10]:
# GENERATE TRAINING DATA;
matplotlib.use('Agg')

# set start at frame_
id_ = 0
features = 10  # Exclude tails
root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/'

# LOAD VIDEO
video_name =root_dir + '/2020-3-16_12_57_12_418305_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

# LOAD CC OUTPUTED TRACES
fname_traces_cc = root_dir + '2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89989.npz'
traces_cc = np.load(fname_traces_cc)
tracesx = traces_cc['tracesx']
print ("tracesx: ", tracesx.shape)
tracesy = traces_cc['tracesy']


# set chain_id for 3D CC at start
chain_id = []
chain_id.append(np.int32(np.arange(tracesx.shape[0]/14))) # start with potentially all the networks from the beginning, 
chain_id_ctr = []
chain_id_ctr.append(np.zeros(tracesx.shape[0]//14,'int32'))
print (chain_id_ctr)

root_dir = root_dir+'/training_images/'   

for chain in chain_id[0]:
    out_fname = root_dir+str(chain)
    try:
        os.mkdir(out_fname)
    except:
        pass
    
from tqdm import tqdm

# FOR CC need to have history, so start with following frame

plotting = False
save = False


# START WITH SECOND FRAME OTHERWISE 
ids = np.arange(1,tracesx.shape[1],1)
frame_start = ids[0]
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_start)
for id_ in tqdm(ids):
    
    ret, frame = original_vid.read()
    chain_id, chain_id_ctr = generate_training_image_cc(tracesx,
                               tracesy,
                               frame,
                               root_dir,
                               id_,
                               features,
                               chain_id,
                               plotting,
                               save,
                               chain_id_ctr)
    
# SAVE CHAIN ID INFORMATION
fname_chain_id = fname_traces_cc[:-4]+"_chain_id.npy"
np.save(fname_chain_id, 
       chain_id)


  0%|          | 10/89988 [00:00<16:06, 93.08it/s]

tracesx:  (70, 89989)
[array([0, 0, 0, 0, 0], dtype=int32)]


100%|██████████| 89988/89988 [27:28<00:00, 54.58it/s] 


In [11]:
# fname_traces_cc = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz'
# fname_chain_id = fname_traces_cc[:-4]+"_chain_id.npy"
# np.save(fname_chain_id, 
#        chain_id)

In [15]:

temp = np.vstack(chain_id)
print (temp[-10:])

[[1157.   nan 1029. 1169.  773.]
 [1157. 1261. 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157. 1262. 1029. 1169.  773.]
 [1157. 1262. 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157. 1263. 1029. 1169.  773.]
 [1157. 1263. 1029. 1169.  773.]]


In [ ]:
# OPTIONAL: RUN ABOVE WITH PLOTTING OFF 
# THE PARSE THE chain_id variable for long chains and only save those

In [ ]:
######################################################################
############## MAKE CLASSIFIER TRAINING DATA FROM SAVED ##############
######################################################################
import numpy as np

root_animals = root_dir + '/training_images_cc/segments_training/'
names = ['female','male','pup1','pup2', 'block']

import glob

data = []
for ctr, name in enumerate(names):
    data.append([])

    fnames = glob.glob(root_animals+name+'/**/*',recursive = True)
    for fname in fnames:
        if '.npy' in fname:
            data[ctr].append(np.load(fname))

for k in range(len(names)):
    print (names[k], len(data[k]))
    temp = np.array(data[k])
    print (temp.shape)
    idx_rand = np.random.choice(np.arange(temp.shape[0]), 
                                #595, 
                                temp.shape[0]//2,
                                replace=False)
    print (k, idx_rand.shape)
    temp = np.array(temp[idx_rand])
    print (temp.shape)
    np.save(root_animals+"id_"+str(k)+'_train.npy', temp)
    
    # save test datasets; 
    temp = np.array(data[k])
    idx = np.arange(temp.shape[0])
    idx_test = np.delete(idx, idx_rand)
    print ("Idx test: ", idx_test.shape)
    
    temp = temp[idx_test]
    print ("saved test: ", temp.shape)
    np.save(root_animals+"id_"+str(k)+'_test.npy', temp)
    
    

female 1042
(1042, 200, 200)
0 (521,)
(521, 200, 200)
Idx test:  (521,)
saved test:  (521, 200, 200)
male 595
(595, 200, 200)
1 (297,)
(297, 200, 200)
Idx test:  (298,)
saved test:  (298, 200, 200)
pup1 1653
(1653, 200, 200)
2 (826,)
(826, 200, 200)
Idx test:  (827,)
saved test:  (827, 200, 200)
pup2 2449
(2449, 200, 200)
3 (1224,)
(1224, 200, 200)
Idx test:  (1225,)
saved test:  (1225, 200, 200)


(100, 100, 50)


In [252]:
# VISUALIZE MATCHES
clrs = ['red','blue','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']


for k in frames:  #skip first match
    print ("frame: ", k)
    im = surfaces_watershed[k]
    idx0 = np.where(im==0)
    #print ("idx0: ", idx0)
    im[idx0]==np.nan
    
    # 
    ids = np.unique(im)
    #print (ids)
    for ctr, id_ in enumerate(ids):
        if id_==0:
            continue

        #
        idx = np.where(im==id_)
        centrex = np.mean(idx[0])
        centrey = np.mean(idx[1])
        print (centrex.shape, centrex)
        
        #centrey = np.mean(im[idx[:,1]])

        match_id = np.array(matches[k][ctr]).squeeze()
        print ("matchid: ", match_id)
        
        plt.scatter(k, centrex+centrey, 
                    c=clrs[match_id%10])
        
plt.show()
        
        

frame:  0
() 20.92491467576792
matchid:  1
() 39.075
matchid:  3
() 62.41414141414141
matchid:  4
() 84.26804123711341
matchid:  5
frame:  1
() 17.873303167420815
matchid:  1
() 30.59722222222222
matchid:  1
() 39.14646464646464
matchid:  3
() 62.38693467336683
matchid:  4
() 84.29473684210527
matchid:  5
frame:  2
() 22.23017902813299
matchid:  1
() 38.775401069518715
matchid:  3
() 62.51269035532995
matchid:  4
() 84.2258064516129
matchid:  5
frame:  3
() 22.31782945736434
matchid:  1
() 38.776595744680854
matchid:  2
() 62.51269035532995
matchid:  4
() 84.23684210526316
matchid:  5
frame:  4
() 22.40776699029126
matchid:  1
() 38.71584699453552
matchid:  2
() 62.759803921568626
matchid:  3
() 84.14778325123153
matchid:  4
frame:  5
() 22.395734597156398
matchid:  1
() 38.73157894736842
matchid:  2
() 62.67464114832536
matchid:  4
() 84.09661835748793
matchid:  5
frame:  6
() 22.27972027972028
matchid:  1
() 39.20873786407767
matchid:  2
() 62.748768472906406
matchid:  4
() 84.317948

TypeError: unsupported operand type(s) for %: 'NoneType' and 'int'

In [215]:
id_ = 145
print (np.unique(surfaces_watershed[id_-1]))
print (np.unique(surfaces_watershed[id_]))
print (matches[id_])


[0 4 6 7]
[0 3 5 6]
[[0], [4], [6], [7]]


In [94]:
##################################################################
#################### EXTRACT PATCHES AND ALIGN THEM ##############
##################################################################

#import skimage.filter


fig=plt.figure()


#img = misc.imread('oval.png', flatten=1)

video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'


original_vid = cv2.VideoCapture(video_name)

frame_id= 900
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = original_vid.read()

img1 = img_array1[frame_id]

aspect = 'equal'
n_rows = np.unique(img1).shape[0]
for ctr_id, id_ in enumerate(np.unique(img1)):
    
    img = img_array1[frame_id].copy()
    
    #id_ = 5
    print ("ctr_ID: ", ctr_id)

    # PLOT FRAME
    ax=plt.subplot(n_rows,5,1+ctr_id*5)
    plt.imshow(frame)
    print (frame.shape)
    
    # PLOT FRAME 
    ax=plt.subplot(n_rows,5,2+ctr_id*5)
    plt.imshow(img, aspect=aspect)

    
    # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
    print (np.unique(img1))
    idx1 = np.where(img1==id_)
    idx2 = np.where(img1!=id_)
    img[idx1]=1
    img[idx2]=0


    width = 125
    padding = 30
    print ("idx1: ", idx1[0],
                    idx1[1])
    centrex = int(np.nanmean(idx1[0]))*8
    centrey = int(np.nanmean(idx1[1]))*8
    print ("cenres:", centrex, centrey,
          centrex//8, centrey//8)

    #frame_patch = np.zeros((200,200),'int32')
    frame_patch = frame[centrex-width-padding:centrex+width+padding, centrey-width-padding:centrey+width+padding][:,:,1]

    ax=plt.subplot(n_rows,5,3+ctr_id*5)
    plt.imshow(frame_patch,aspect=aspect)
    
    
    #
    y, x = np.nonzero(img)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])

    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)

    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]


    # PLOT THE PATCH WITH ANIMAL IN IT
    ax=plt.subplot(n_rows,5,4+ctr_id*5)
    scale = 20
    plt.plot([x_v1*-scale*2, x_v1*scale*2],
             [y_v1*-scale*2, y_v1*scale*2], color='red')
    plt.plot([x_v2*-scale, x_v2*scale],
             [y_v2*-scale, y_v2*scale], color='blue')
    plt.plot(x, y, 'k.')
    plt.axis('equal')
    plt.gca().invert_yaxis()  # Match the image system with origin at top left

    #fig=plt.figure()

    theta = np.arctan((x_v1)/(y_v1))  
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])
    transformed_mat = rotation_mat * coords

    # plot the transformed blob
    x_transformed, y_transformed = transformed_mat.A
    plt.plot(x_transformed, y_transformed, 'g.')


    # ROTATE RAW DATA
    ax=plt.subplot(n_rows, 5,5+ctr_id*5)
    try: 
        frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
        frame_rotated = frame_rotated[padding:-padding, padding: -padding]
        plt.imshow(frame_rotated,aspect=aspect)
    except:
        pass
plt.show()


IndexError: list index out of range

In [227]:
##################################################################
#################### REMOVE PATCHES BELOW THRESHOLD ##############
##################################################################

centres = []
ids = []
min_size = 0
for k in range(102):
    idx = np.unique(img_array[k])
    centres.append([])
    ids.append([])
    #print (k, idx.shape, idx)
    ctr=0
    for id_ in idx:
        idx2 = np.where(img_array[k]==id_)
        if idx2[0].shape[0]< min_size:
            img_array[k][idx2]=0
            continue
        ids[k].append(ctr)
        if idx2[0].shape[0]>1:
            centres[k].append(np.array([np.mean(idx2[0]), np.mean(idx2[1])]))
        else:
            centres[k].append(np.array([idx2[0], idx2[1]]))
        ctr+=1    
            
        #print (centres[k])
    #print('')

In [230]:
# LINK CENTRES ACROSS TIME
centres = np.array(centres)
print (centres.shape)
from scipy.spatial import distance

matches = []
for k in range(1,102,1):
    
    # find min distances 
    dist = distance.cdist(centres[k],centres[k-1]) 
    print (k, dist.min(axis=1))
    match = dist.argmin(axis=1)
    print ("k, ", k, "  match: ",  match)
    matches.append(match)
    
matches = np.array(matches)
print (matches.shape)

(102,)


ValueError: could not broadcast input array from shape (2,1) into shape (2)

In [216]:
# FIND CENTRES FOR ALL THE CONTOURS
#centres = []
#min_size = 100
imgs = img_array.copy()
fig=plt.figure(figsize=(20,20))
ctr_plot=0
for k in range(20,40,1):
    ax = plt.subplot(4,5,ctr_plot+1)
    #idx = np.unique(img_array[k])
    #print (k, idx.shape, idx)
    ctr=0
    for id_ in ids[k]:
        idx2 = np.where(img_array[k]==id_)
        #if idx2[0].shape[0]< min_size:
        #    continue
        print ("frame: ", k, " animal id: ", id_, " ctr: ", ctr)
        print ("ids ", ids[k])
        print ("matches ", matches[k])
        print ("   matches ", matches[k-1][ctr])
        #print (matches[k])
        imgs[k][idx2]=matches[k-1][ctr]
        ctr+=1
        
    plt.ylabel(str(k))
    idx =np.where(imgs[k]<0)
    imgs[k][idx]=0
    
    vmin = np.min(imgs[k]); vmax=np.max(imgs[k])
    img2 = surfaces[k]
    img2 = (img2-np.min(img2))/(np.max(img2)-np.min(img2))
    img2 = img2*(vmax-vmin)+vmin
    img_final = np.vstack((imgs[k],img2))
    img_final[img_final.shape[0]//2]=np.nan
    plt.imshow(img_final)
    plt.xticks([])
    plt.yticks([])
    ctr_plot+=1
    
#plt.show()
#"/home/cat/watershed.png",dpi=1200)
fig.savefig("/home/cat/watershed.svg", format='svg', dpi=300)
plt.close()

frame:  20  animal id:  0  ctr:  0
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  0
frame:  20  animal id:  1  ctr:  1
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  2
frame:  20  animal id:  2  ctr:  2
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  2
frame:  20  animal id:  3  ctr:  3
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  3
frame:  20  animal id:  4  ctr:  4
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  5
frame:  20  animal id:  5  ctr:  5
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  6
frame:  21  animal id:  0  ctr:  0
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  0
frame:  21  animal id:  1  ctr:  1
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  2
frame:  21  animal id:  2  ctr:  2
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  2
frame:  21  animal id:  3  ctr:  3
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4

In [176]:
frame = 24
ax =plt.subplot(1,2,1)
plt.imshow(img_array[frame])
ax =plt.subplot(1,2,2)
plt.imshow(surfaces[frame])

plt.show()

In [25]:
from matplotlib import pyplot as plt
from celluloid import Camera

fig = plt.figure()
camera = Camera(fig)
for i in range(100):
    
    plt.imshow(surfaces[i])
    camera.snap()
animation = camera.animate()
animation.save('/home/cat/surfaces.mp4',fps=30)

In [21]:
x_train = np.load('/home/cat/x_train.npy')
print (x_train.shape)

for k in range(10):
    ax=plt.subplot(2,5,k+1)
    print (x_train[k*27])
    plt.imshow(np.int32(x_train[k*27]))
plt.show()

(2000, 34, 34, 3)
[[[  6.   6.   6.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [103. 103. 103.]
  [ 57.  57.  57.]
  [  5.   5.   5.]]

 [[ 14.  14.  14.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [ 65.  65.  65.]
  [ 81.  81.  81.]
  [ 32.  32.  32.]]

 [[ 18.  18.  18.]
  [ 20.  20.  20.]
  [ 19.  19.  19.]
  ...
  [ 59.  59.  59.]
  [ 64.  64.  64.]
  [ 21.  21.  21.]]

 ...

 [[ 30.  30.  30.]
  [ 14.  14.  14.]
  [ 14.  14.  14.]
  ...
  [135. 135. 135.]
  [144. 144. 144.]
  [ 43.  43.  43.]]

 [[ 74.  74.  74.]
  [ 66.  66.  66.]
  [ 40.  40.  40.]
  ...
  [145. 145. 145.]
  [130. 130. 130.]
  [  6.   6.   6.]]

 [[  2.   2.   2.]
  [ 32.  32.  32.]
  [ 26.  26.  26.]
  ...
  [ 48.  48.  48.]
  [ 29.  29.  29.]
  [  0.   0.   0.]]]
[[[  6.   6.   6.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [103. 103. 103.]
  [ 57.  57.  57.]
  [  5.   5.   5.]]

 [[ 14.  14.  14.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [ 65.  65.  65.]
  [ 81.  81.  81.]
  [ 32.  32.  32.]]



In [27]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/1467/predictions.npz')
confidence = data['confidence']
predictions = data['predictions']
print (confidence)
print (predictions)

print (np.unique(predictions,return_counts=True))

[[0.00771165 0.87360513 0.01278432 0.10589889]
 [0.00548172 0.86633325 0.01190533 0.1162797 ]
 [0.01268774 0.7352125  0.01571083 0.23638895]
 ...
 [0.0023764  0.91065294 0.00576707 0.08120355]
 [0.02976477 0.74782044 0.02765195 0.19476283]
 [0.00231505 0.9284846  0.00448818 0.06471218]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 